In [4]:
import numpy as np
import cv2
import sys
import os
import pickle
import pandas as pd

from src import RESULT_DIR
from src.utils import MCC, DIS


# Directories
output_dir = os.path.join(RESULT_DIR, 'test_results')
tables_dir = os.path.join(output_dir, 'tables')

In [16]:
def get_table(dbase, level, ground_truth):
    results_csv = os.path.join(tables_dir, 
                               '{0}_full_results.csv'.format(dbase))

    # Columns
    suffix = '_{0}{1}'.format(level[0], ground_truth[0]) 
    columns = [k + suffix 
            for k in ['gt', 'tn', 'fp', 'fn', 'tp']]
    all_columns = ['video', 'frame', 'fold'] + columns
    cc = all_columns[4:]
    
    res_df = pd.read_csv(results_csv, index_col=0)
    res_df = res_df[all_columns]
   # res_df = res_df.groupby([vid_fold]).sum().loc[:, cc]
    res_df['MCC'] = res_df.apply(lambda row: MCC(row[cc[0]], row[cc[1]],
                                                row[cc[2]], row[cc[3]]),
                                 axis=1)
    res_df['DIS'] = res_df.apply(lambda row: DIS(row[cc[0]], row[cc[1]],
                                                row[cc[2]], row[cc[3]]),
                                 axis=1)

    res_df['TPR'] =  res_df['tp'+suffix]/(res_df['tp'+suffix]+res_df['fn'+suffix])
    res_df['FPR'] =  res_df['fp'+suffix]/(res_df['fp'+suffix]+res_df['tn'+suffix])

    sum_df = res_df.sum()

    res_df.loc['Sum', :] = sum_df
    res_df.loc['Average', :] = res_df.mean()
    res_df.loc['Std. deviation', :] = res_df.std()
    res_df.loc['Overall', 'MCC'] = MCC(sum_df[cc[0]], sum_df[cc[1]],
                                    sum_df[cc[2]], sum_df[cc[3]])
    res_df.loc['Median', :] = res_df.median()
    res_df.loc['Overall', 'DIS'] = DIS(sum_df[cc[0]], sum_df[cc[1]],
                                    sum_df[cc[2]], sum_df[cc[3]])
    res_df.loc['Overall', 'TPR'] = sum_df['tp'+suffix]/(sum_df['tp'+suffix]+sum_df['fn'+suffix])
    res_df.loc['Overall', 'FPR'] = sum_df['fp'+suffix]/(sum_df['fp'+suffix]+sum_df['tn'+suffix])
    res_df = res_df[['tn'+suffix, 'fp'+suffix, 
                     'fn'+suffix, 'tp'+suffix, 'MCC', 'DIS',
                     'TPR', 'FPR']]
    return res_df


#res_df = get_table(dbase, level, ground_truth)

In [31]:
dbase = 'LightGBM'
level = 'object'
ground_truth = 'silhouette'
vid_fold = 'video'

# dbases = ['DAOMC', 'ADMULT', 'MCBS_Both', 'mcDTSR', 'TCF_LMO',
#           'LightGBM_temporal','LightGBM_warp', 'mcDTSR_3', 'mcDTSR_4', 
#           'mcDTSR_5', 'mcDTSR_6']

dbases = ['DAOMC', 'ADMULT', 'MCBS_Both', 'mcDTSR', 'TCF_LMO',
          'LightGBM_temporal','LightGBM_warp']

results = {d:get_table(d, level, ground_truth) for d in dbases}


# concatenate the DataFrames horizontally
df_concat = pd.concat(results.values(), axis=1)

# create a multi-level column index using the dictionary keys
columns = pd.MultiIndex.from_tuples([(key, col) for key in results.keys()
                                     for col in results[key].columns])

# assign the multi-level column index to the concatenated DataFrame
df_concat.columns = columns
df_concat = df_concat.loc[df_concat.index.isin(['Average', 'Std. deviation', 
                                                'Median', 'Overall']), :]

In [32]:
df_mcc = df_concat.loc[:, (slice(None), 'MCC')]
df_dis = df_concat.loc[:, (slice(None), 'DIS')]
df_tpr = df_concat.loc[:, (slice(None), 'TPR')]
df_fpr = df_concat.loc[:, (slice(None), 'FPR')]

df_mcc = df_mcc.droplevel(level=1, axis=1)
df_dis = df_dis.droplevel(level=1, axis=1)

In [33]:
df_tpr

,DAOMC,ADMULT,MCBS_Both,mcDTSR,TCF_LMO,LightGBM_temporal,LightGBM_warp
,TPR,TPR,TPR,TPR,TPR,TPR,TPR
Average,1.005526,1.409394,0.434237,1.664411,1.916154,1.941204,1.690075
Std. deviation,46.851446,65.667115,18.789974,77.028246,88.889936,89.669288,72.210845
Overall,0.502821,0.704778,0.217147,0.832302,0.958188,0.970716,0.845153
Median,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
df_fpr

,DAOMC,ADMULT,MCBS_Both,mcDTSR,TCF_LMO,LightGBM_temporal,LightGBM_warp
,FPR,FPR,FPR,FPR,FPR,FPR,FPR
Average,0.411056,1.314350,1.994743,1.999831,1.812472,1.998819,0.681606
Std. deviation,11.987514,51.802410,108.296352,108.609395,85.403318,108.784228,24.936753
Overall,0.205588,0.657281,0.997456,1.000000,0.906338,0.999494,0.340867
Median,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [35]:
print("\\begin{table}[ht!]\n\centering\n\\begin{adjustbox}{width=\columnwidth,center}\n")
print(df_tpr.to_latex())
print("\end{adjustbox}\n\end{table}")

\begin{table}[ht!]
\centering
\begin{adjustbox}{width=\columnwidth,center}

\begin{tabular}{lrrrrrrr}
\toprule
{} &      DAOMC &     ADMULT &  MCBS\_Both &     mcDTSR &    TCF\_LMO & LightGBM\_temporal & LightGBM\_warp \\
{} &        TPR &        TPR &        TPR &        TPR &        TPR &               TPR &           TPR \\
\midrule
Average        &   1.005526 &   1.409394 &   0.434237 &   1.664411 &   1.916154 &          1.941204 &      1.690075 \\
Std. deviation &  46.851446 &  65.667115 &  18.789974 &  77.028246 &  88.889936 &         89.669288 &     72.210845 \\
Overall        &   0.502821 &   0.704778 &   0.217147 &   0.832302 &   0.958188 &          0.970716 &      0.845153 \\
Median         &   1.000000 &   1.000000 &   0.000000 &   1.000000 &   1.000000 &          1.000000 &      1.000000 \\
\bottomrule
\end{tabular}

\end{adjustbox}
\end{table}


/tmp/ipykernel_5825/3981045942.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_tpr.to_latex())


In [36]:
print("\\begin{table}[ht!]\n\centering\n\\begin{adjustbox}{width=\columnwidth,center}\n")
print(df_fpr.to_latex())
print("\end{adjustbox}\n\end{table}")

\begin{table}[ht!]
\centering
\begin{adjustbox}{width=\columnwidth,center}

\begin{tabular}{lrrrrrrr}
\toprule
{} &      DAOMC &     ADMULT &   MCBS\_Both &      mcDTSR &    TCF\_LMO & LightGBM\_temporal & LightGBM\_warp \\
{} &        FPR &        FPR &         FPR &         FPR &        FPR &               FPR &           FPR \\
\midrule
Average        &   0.411056 &   1.314350 &    1.994743 &    1.999831 &   1.812472 &          1.998819 &      0.681606 \\
Std. deviation &  11.987514 &  51.802410 &  108.296352 &  108.609395 &  85.403318 &        108.784228 &     24.936753 \\
Overall        &   0.205588 &   0.657281 &    0.997456 &    1.000000 &   0.906338 &          0.999494 &      0.340867 \\
Median         &   0.000000 &   1.000000 &    1.000000 &    1.000000 &   1.000000 &          1.000000 &      0.000000 \\
\bottomrule
\end{tabular}

\end{adjustbox}
\end{table}


/tmp/ipykernel_5825/1175515453.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_fpr.to_latex())


In [72]:
# Create the style object
df_style = df_mcc.style.apply(
    lambda x: ['font-weight: bold' if v == x[x.index.max()] else '' for v in x],
    axis=1
).apply(
    lambda x: ['font-weight: bold' if v == x[x.index.min()] else '' for v in x],
    axis=1, subset=pd.IndexSlice['Std. deviation', :]
)

# Convert to LaTeX
latex_table = df_style.to_latex()

print(latex_table)


\begin{tabular}{lrrrrrrr}
 & DAOMC & ADMULT & MCBS_Both & mcDTSR & TCF_LMO & LightGBM_temporal & LightGBM_warp \\
video &  &  &  &  &  &  &  \\
Average & 0.380501 & 0.457119 & 0.000558 & \font-weightbold -0.014802 & 0.030048 & 0.010230 & 0.623659 \\
Std. deviation & 0.283957 & \font-weightbold 0.201435 & 0.027383 & \font-weightbold 0.043969 & 0.085761 & 0.045464 & 0.270707 \\
Overall & 0.604219 & 0.584970 & 0.006895 & \font-weightbold -0.014536 & 0.064795 & 0.052447 & 0.772486 \\
Median & 0.460846 & 0.477773 & -0.007479 & \font-weightbold 0.000000 & \font-weightbold 0.000000 & \font-weightbold 0.000000 & 0.721401 \\
\end{tabular}

